this takes in the raw E-Prime files, which have been concatonated together into one large file (raw_concat.csv), and does minimal processing 

In [170]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt

In [171]:
#this makes it so that the WHOLE dataframe prints, just fyi, this can bog down your code

pd.set_option('display.max_columns', 1000)  # or 1000
pd.set_option('display.max_rows', None)  # or 1000
pd.set_option('display.max_colwidth', 199)  # or 199

In [ ]:
incomplete_runs_subj = [] 
SST_concat = pd.DataFrame()
incomplete_runs_df = pd.DataFrame()
raw_concat = raw_concat[np.logical_and(raw_concat.TrialCode != 'BeginFix', \
                                       raw_concat.TrialCode != 'EndFix')]

raw_concat = raw_concat[~raw_concat['TrialCode'].isnull()]

for i in raw_concat['NARGUID'].unique(): 
    sub_df = raw_concat.loc[raw_concat['NARGUID'] == i]
    #add TrialNum col from 1-360
    sub_df['TrialNum'] = np.arange(1, len(sub_df)+1)
#     stop_trial_mask = (sub_df['TrialCode'] == 'IncorrectStop') | \
#                       (sub_df['TrialCode'] == 'CorrectStop')
#     stop_trial_idx = stop_trial_mask[stop_trial_mask == True].index
#     sub_df['StopTrial'] = ""
#     sub_df['StopTrial'][stop_trial_idx] = np.arange(1, len(sub_df.loc[stop_trial_idx])+ 1)
    if len(~sub_df['TrialNum'].isnull()) == 360: 
        if 'StopTooEarly' not in sub_df['TrialCode'].unique(): 
            SST_concat = SST_concat.append(sub_df)
    else: 
        incomplete_runs_df = incomplete_runs_df.append(sub_df)
        incomplete_runs_subj.append(i)
    

incomplete_runs_df.to_csv('incomplete_subjects.csv')

In [ ]:
#this saves as a csv so you can avoid running that loops multiple times 
#SST_concat.to_csv('reran_partially_cleaned.csv')

In [172]:
#this reads in the file above, to avoid running that loop 
SST_concat = pd.read_csv('reran_partially_cleaned.csv')

/Users/work/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (2,14,37,43,52,81,104,114,115) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


### Check "TrialCode" accuracy

In [173]:
#some subjects have their subtrial under Procedure[Trial]
SST_concat['Procedure[SubTrial]'].loc[SST_concat['Procedure[SubTrial]'].isnull()] = SST_concat['Procedure[Trial]'] 
#the trial type column has inconsistent notation for stop trials 
SST_concat['trial_type'] = SST_concat['Procedure[SubTrial]'].replace('VariableStopTrial.*', 'StopTrial', regex=True)

/Users/work/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:205: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


In [174]:
#find all go trials 
go_trial_mask = SST_concat.loc[SST_concat['Procedure[SubTrial]'] == 'GoTrial']
go_trial_idx = go_trial_mask[go_trial_mask==True].index

In [175]:
#the response recordings are inconsistent in type (str, int, float) this fixes that 
cresp_replace = {'2.0': 2.0,
                 '1.0': 1.0,
                 '3.0': 3.0,
                 '4.0': 4.0,
                 '1,{LEFTARROW}': 1.0,
                 '2,{RIGHTARROW}': 2.0}

resp_replace = {'2.0': 2.0,
                '1.0': 1.0,
                '3.0': 3.0,
                '4.0': 4.0,
                '{LEFTARROW}': 1.0,
                '{RIGHTARROW}': 2.0}

SST_concat['Go.RESP'].replace(to_replace = resp_replace, inplace = True)
SST_concat['Go.CRESP'].replace(to_replace = cresp_replace, inplace=True)

In [176]:
#create my own correct response column 
SST_concat['correct_go_response'] = np.NaN

SST_concat['correct_go_response'].loc[(~SST_concat['Go.RESP'].isnull()) & 
                                      (SST_concat['Go.CRESP'] == SST_concat['Go.RESP'])] = float(1)

SST_concat['correct_go_response'].loc[(SST_concat['Go.RESP'].isnull()) & 
                                      (SST_concat['Go.CRESP'] == SST_concat['Fix.RESP'])] = float(1)

SST_concat['correct_go_response'].loc[(~SST_concat['Go.RESP'].isnull()) & 
                                      (SST_concat['Go.CRESP'] != SST_concat['Go.RESP']) &
                                      (SST_concat['trial_type'] == 'GoTrial')] = float(0)


SST_concat['correct_go_response'].loc[(SST_concat['Go.RESP'].isnull()) & 
                                      (SST_concat['Go.CRESP'] != SST_concat['Fix.RESP']) &
                                      (SST_concat['trial_type'] == 'GoTrial')] = float(0)


SST_concat['correct_go_response'].loc[(SST_concat['Go.RESP'].isnull()) & (SST_concat['Fix.RESP'].isnull()) & 
                                      (SST_concat['trial_type'] == 'GoTrial')] = 'omission'




/Users/work/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:205: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


In [177]:
#check that I only have ones, zeros, and omissions
SST_concat.loc[SST_concat['trial_type'] == 'GoTrial']['correct_go_response'].unique() 

array([1.0, 'omission', 0.0], dtype=object)

In [178]:
#correct stop column 
SST_concat['correct_stop'] = np.NaN

SST_concat['correct_stop'].loc[(SST_concat['StopSignal.RESP'].isnull()) & (SST_concat['Fix.RESP'].isnull()) & \
                               (SST_concat['trial_type'] == 'StopTrial') & (SST_concat['SSD.RESP'].isnull())] = float(1)

SST_concat['correct_stop'].loc[(~(SST_concat['StopSignal.RESP'].isnull()) | ~(SST_concat['Fix.RESP'].isnull()) \
                                | ~(SST_concat['SSD.RESP'].isnull())) & (SST_concat['trial_type'] == 'StopTrial')] = float(0)

/Users/work/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:205: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


In [179]:
SST_concat['correct_stimulus_mapping_1'] = np.NaN
SST_concat['correct_stimulus_mapping_2'] = np.NaN

In [180]:
SST_concat.loc[SST_concat['Stimulus'] == 'images/Right_Arrow.bmp', 'correct_stimulus_mapping_1'] = SST_concat.loc[SST_concat['Stimulus'] == 'images/Right_Arrow.bmp']['Go.CRESP'].dropna().unique()[0]
SST_concat.loc[SST_concat['Stimulus'] == 'images/Right_Arrow.bmp', 'correct_stimulus_mapping_2'] = SST_concat.loc[SST_concat['Stimulus'] == 'images/Right_Arrow.bmp']['Go.CRESP'].dropna().unique()[1]

SST_concat.loc[SST_concat['Stimulus'] == 'images/Left_Arrow.bmp', 'correct_stimulus_mapping_1'] = SST_concat.loc[SST_concat['Stimulus'] == 'images/Left_Arrow.bmp']['Go.CRESP'].dropna().unique()[0]
SST_concat.loc[SST_concat['Stimulus'] == 'images/Left_Arrow.bmp', 'correct_stimulus_mapping_2'] = SST_concat.loc[SST_concat['Stimulus'] == 'images/Left_Arrow.bmp']['Go.CRESP'].dropna().unique()[1]

In [181]:
#correct stop choice response 
SST_concat['correct_stop_mapping'] = np.NaN 
SST_concat['correct_stop_mapping'].loc[(SST_concat['correct_stop'] == 0) & (~SST_concat['SSD.RESP'].isnull()) &\
                                       ((SST_concat['SSD.RESP'] == SST_concat['correct_stimulus_mapping_1']) | (SST_concat['SSD.RESP'] == SST_concat['correct_stimulus_mapping_2']))] = float(1)

SST_concat['correct_stop_mapping'].loc[(SST_concat['correct_stop'] == 0) & (~SST_concat['SSD.RESP'].isnull()) &\
                                      (SST_concat['SSD.RESP'] != SST_concat['correct_stimulus_mapping_1']) & (SST_concat['SSD.RESP'] != SST_concat['correct_stimulus_mapping_2'])] = float(0)


/Users/work/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:205: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


In [182]:
SST_concat['correct_stop_mapping'].loc[(SST_concat['correct_stop'] == 0) & (SST_concat['SSD.RESP'].isnull()) & (~SST_concat['StopSignal.RESP'].isnull()) &\
                                       ((SST_concat['StopSignal.RESP'] == SST_concat['correct_stimulus_mapping_1']) | (SST_concat['StopSignal.RESP'] == SST_concat['correct_stimulus_mapping_2']))] = float(1)

SST_concat['correct_stop_mapping'].loc[(SST_concat['correct_stop'] == 0) & (SST_concat['SSD.RESP'].isnull()) & (~SST_concat['StopSignal.RESP'].isnull()) &\
                                       ((SST_concat['StopSignal.RESP'] != SST_concat['correct_stimulus_mapping_1']) & (SST_concat['StopSignal.RESP'] != SST_concat['correct_stimulus_mapping_2']))] = float(0)

/Users/work/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:205: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


In [183]:
SST_concat['correct_stop_mapping'].loc[(SST_concat['correct_stop'] == 0) & (SST_concat['SSD.RESP'].isnull()) & (SST_concat['StopSignal.RESP'].isnull()) & (~SST_concat['Fix.RESP'].isnull()) &\
                                       ((SST_concat['Fix.RESP'] == SST_concat['correct_stimulus_mapping_1']) | (SST_concat['Fix.RESP'] == SST_concat['correct_stimulus_mapping_2']))] = float(1)

SST_concat['correct_stop_mapping'].loc[(SST_concat['correct_stop'] == 0) & (SST_concat['SSD.RESP'].isnull()) & (SST_concat['StopSignal.RESP'].isnull()) & (~SST_concat['Fix.RESP'].isnull()) &\
                                       ((SST_concat['Fix.RESP'] != SST_concat['correct_stimulus_mapping_1']) & (SST_concat['Fix.RESP'] != SST_concat['correct_stimulus_mapping_2']))] = float(0)

/Users/work/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:205: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


### fix the go.rt calculation

since the trial unfolds over several different sub trials, it's necessary to add some of these together to get correct rt

In [184]:
#find the index of where there's a fixation response on go trials 
go_fix_resp = (~SST_concat['Fix.RESP'].isnull()) & \
              (SST_concat['Go.RESP'].isnull()) & \
              (SST_concat['trial_type'] == 'GoTrial')
go_fix_idx = go_fix_resp[go_fix_resp == True].index

In [185]:
#make column from Go.RT values
SST_concat['go_rt_adjusted'] = SST_concat['Go.RT'].copy()
#for long response trials, add the fix.rt to the go.duration
SST_concat['go_rt_adjusted'][go_fix_idx] = SST_concat.loc[go_fix_idx]['Go.Duration'] +  \
                                           SST_concat.loc[go_fix_idx]['Fix.RT']

/Users/work/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


#### fix the stop sig rt calculation

In [186]:
#find all stop trials
stop_trial_mask = (SST_concat['trial_type'] == 'StopTrial')
#make an index of all stop trials 
stop_trial_idx = stop_trial_mask[stop_trial_mask == True].index

In [187]:
SST_concat['stop_rt_adjusted'] = SST_concat['StopSignal.RT']
#find all stop failure trials with resp during fix.resp, and no resp during stopsignal.resp
stop_fix_resp = SST_concat['Fix.RESP'].notnull() & \
                SST_concat['StopSignal.RESP'].isnull() & \
                ((SST_concat['trial_type'] == 'StopTrial') & (SST_concat['correct_stop'] == 0))

stop_fix_idx = stop_fix_resp[stop_fix_resp == True].index

stop_SSD_resp = SST_concat['SSD.RESP'].notnull() & (SST_concat['trial_type'] == 'StopTrial')

In [188]:
#use that index to add stop signal duration and stop signal duration to get correct stop signal RT on these trials
SST_concat['stop_rt_adjusted'][stop_fix_resp] = SST_concat.loc[stop_fix_resp]['StopSignal.Duration'] +\
                                           SST_concat.loc[stop_fix_resp]['Fix.RT'] + \
                                         SST_concat[stop_fix_resp]['SSDDur']

SST_concat['stop_rt_adjusted'][stop_SSD_resp] = SST_concat.loc[stop_SSD_resp]['SSD.RT']

/Users/work/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/Users/work/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [189]:
#find the stop failure trials and add the stop signal duration for correct stop fail RT 

##erroroneous : 
# stop_resp_mask = (SST_concat['StopSignal.RESP'].notnull() &  \
#                              SST_concat['SSD.RESP'].isnull() & \
#                 ~SST_concat['stop_rt_adjusted'].isnull())

stop_resp_mask = ((SST_concat['StopSignal.RESP'].notnull()  \
                & SST_concat['SSD.RESP'].isnull()) & \
                 (SST_concat['trial_type'] == 'StopTrial'))
        
stop_resp_idx = stop_resp_mask[stop_resp_mask == True].index

#add SSDDur to response time 
SST_concat['stop_rt_adjusted'][stop_resp_idx] = \
        SST_concat['stop_rt_adjusted'][stop_resp_idx] + SST_concat['SSDDur'][stop_resp_idx]

/Users/work/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.


### create go stimuli duration

In [190]:
#create column for Go.Stim.Duration - when there's no go response, this should be 1000ms
SST_concat['go_stim_duration'] = SST_concat['Go.RT'].copy()
SST_concat['go_stim_duration'].loc[SST_concat['Go.RT'] == 0] = SST_concat['Go.Duration'].loc[SST_concat['Go.RT'] == 0]

SST_concat['go_stim_duration'].loc[~(SST_concat['SSD.RESP'].isnull())] = SST_concat.loc[~SST_concat['SSD.RESP'].isnull()]['SSD.RT']
SST_concat['go_stim_duration'].loc[(SST_concat['SSD.RT']  == 0)] = SST_concat.loc[SST_concat['SSD.RT'] == 0]['SSDDur']


/Users/work/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:205: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


In [191]:
#this saves it to csvs that are small enough to be viewed by excel
#SST_concat.iloc[:867600].to_csv('SST_cleaned_7231_1.csv')
#SST_concat.iloc[867600:1735560, :].to_csv('SST_cleaned_7231_2.csv')
#SST_concat.iloc[1735560:].to_csv('SST_cleaned_7231_3.csv')

In [192]:
SST_concat.to_csv('SST_cleaned_7231_all_rows_all_columns.csv')